In [ ]:
import inspect
from functools import wraps
from warnings import warn

In [ ]:
def decoder(func):
    """Decode bytes objects.

    Note:
        This function is intended to be used as a decorator like follows.
        >>> @decoder
        >>> def func(*args, **kwargs):
        >>>     # Do something.
        >>>     return ret

    Args:
        func (function) A function to be wrapped.

    Return:
        wrapper (function): A wrapped function.
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        from warnings import warn

        ret = func(*args, **kwargs)
        if isinstance(ret, bytes):
            ret = ret.decode()
        elif isinstance(ret, list):
            if all(isinstance(elem, bytes) for elem in ret):
                ret = [elem.decode() for elem in ret]
        else:
            warn(
                f'Return type of wrapped function: {type(ret)}' + \
                f'You should not use this decorator.'
            )

        return ret
    return wrapper


class Check(object):
    """Check the value of specified argument.

    This is the base class of the value checker.

    Note:
        This class itself is not used, but it is inherited and used
        by a child classe the function as a decorator.

    Args:
        name (str): Name of the specified argument.
    """
    def __init__(self, name):
        self.name = name

    def __call__(self, func):
        """Function like a decorator.

        Note:
            This method is overrided "__call__" to function as a decorator.

        Args:
            func (function) A function to be wrapped.

        Return:
            wrapper (function): A wrapped function.
        """
        sig = inspect.signature(func)

        @wraps(func)
        def wrapper(*args, **kwargs):
            params = sig.parameters
            for i, (key, val) in enumerate(params.items()):
                if val.kind is not inspect.Parameter.POSITIONAL_OR_KEYWORD:
                    break
                try:
                    kwargs.update({key: args[i]})
                except IndexError:
                    kwargs.setdefault(key, val.default)

            if self.name in kwargs.keys():
                self._check(kwargs)
            else:
                raise KeyError(
                    f'"{func.__name__}" does not have the argument "{self.name}"'
                )

            return func(**kwargs)
        return wrapper

    def _check(self, kwargs_):
        """Check the value to set a device.

        This method is the base method to check the value of specified
        argument.
        This method is used in "__call__" method.

        Note:
            You override this method in child class.

        Args:
            kwargs_ (dict): Arbitrary keyword arguments.
        """
        pass


class filter(Check):
    """Filter the setting value.

    This class is based on 'Check'

    Note:
        This class is intended to be used as a decorator like follows.
        >>> @filter('name', 0.01, 4.99)
        >>> def func(name, *args, **kwargs):
        >>>     # Do something.
        >>>     return
    """
    def __init__(self, name, min_, max_):
        """Initialize 'filter'.

        Args:
            name (str): Name of the specified argument.
            min_ (int or float): Minimum value of setting to a device.
            max_ (int or float): Maximum value of setting to a device.
        """
        super().__init__(name)
        self.min = min_
        self.max = max_

    def _check(self, kwargs_):
        """Check the value to set a device.

        Note:
            This method is overrided "_check" in the base class.

        Args:
            kwargs_ (dict): Arbitrary keyword arguments.

        Return:
            None
        """
        if not isinstance(kwargs_[self.name], (int, float)):
            raise TypeError(f'"{self.name}" must be int or float.')

        value = kwargs_[self.name]
        if not self.min <= value <= self.max:
            warn(
                f'\n\tSet value: {value}' + \
                f'\n\tYou should set "{self.name}" {self.min} - {self.max}.'
            )
            kwargs_[self.name] = self.min if value < self.min else self.max
        return


class increments(Check):
    """Check and modify the increments of the value to set a device.

    This class is based on 'Check'.

    Note:
        This class is intended to be used as a decorator like follows.
        >>> @increments('name', 0.5)
        >>> def func(name, *args, **kwargs):
        >>>     # Do something.
        >>>     return
    """
    def __init__(self, name, inc):
        """Initialize 'increments'.

        Args:
            name (str): Name of the specified argument.
            inc (int or float): The increments of the value to set a device.
        """
        super().__init__(name)
        self.inc = inc

    def _check(self, kwargs_):
        """Check the value to set a device.

            Note:
                This method is overrided "_check" in the base class.

            Args:
                kwargs_ (dict): Arbitrary keyword arguments.

            Return:
                None
        """
        if not isinstance(kwargs_[self.name], (int, float)):
            raise TypeError(f'"{self.name}" must be int or float.')

        value = kwargs_[self.name]
        reminder = value % self.inc
        if not reminder == 0:
            warn(
                f'\n\tSet value: {value}' \
                f'\n\tYou should set "{self.name}" in multiples of {self.inc}.'
            )
            if reminder <= self.inc / 2:
                kwargs_[self.name] = value - reminder
            else:
                kwargs_[self.name] = value - reminder + self.inc
        return

In [ ]:
@filter('vadj', 0, 16_383_999_000)
@increments('vadj', 1000)
def set_vadj(vadj):
    print(f'{vadj}')
    return

@decoder
def show_status():
    ret = f'status'
    return ret

In [ ]:
set_vadj(16_383_999_000 - 1)

In [ ]:
type(show_status())

In [ ]:
import inspect

def aaa(a, b, c, *args, **kwargs):
    print(f'positional: {a}, {b}, {c}')
    print(f'args: {args}')
    print(f'kwargs: {kwargs}')
    return

In [ ]:
aaa(3, 2, 'ddd', 1, 2, 3, x=2, y=4)

In [ ]:
sig = inspect.signature(aaa(10, 4.323, 'ddd', 1, 2, 3, x=4, y=5))

In [ ]:
params = sig.parameters

In [ ]:
for i, (key, val) in enumerate(params.items()):
    print(i)
    print(key, val)